实验计划

模型架构mlp+softmax多分类

1.实现pytorch版,并且稍微调参

2.复刻myTorch版

3.基于myTorch做消融实验(优化器，初始化策略)

In [10]:
# 用于记录每个单元格的运行时间

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.03 ms (started: 2024-11-04 15:43:15 +08:00)


In [11]:
#把项目目录添加至环境变量
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()

# 获取上级目录
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
print(parent_dir)
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)



/root/autodl-tmp/myTorch
time: 2.25 ms (started: 2024-11-04 15:43:15 +08:00)


In [20]:
# 基于pytorch导入数据
import torch
from torchvision import datasets, transforms
from torch.utils.data import random_split
import MyTorch.Dataloader
data_dir = "../../dataset"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
data_transforms = transforms.Compose([
    transforms.ToTensor(),  # 转换为Tensor
    transforms.Normalize((0.5,), (0.5,)),  # 归一化，均值为0.5，标准差为0.5
])
# MNIST dataset
train_data_full = datasets.MNIST(data_dir, train=True, download=True, transform=data_transforms)
test_data = datasets.MNIST(data_dir, train=False, download=True, transform=data_transforms)

print(f"shape:{train_data_full.data.shape}")
print(f"mean:{train_data_full.data.float().mean()}")#pytorch dataset的transform只会在取数据进入model时调用，故此时print仍然是原始数据

val_size = int(0.1 * len(train_data_full))
train_data, val_data = random_split(train_data_full, [len(train_data_full) - val_size, val_size])
print("train_data:", len(train_data))
print("val_data:", len(val_data))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device:{device}")


shape:torch.Size([60000, 28, 28])
mean:33.31842041015625
train_data: 54000
val_data: 6000
device:cuda
time: 112 ms (started: 2024-11-04 15:56:43 +08:00)


In [13]:
#定义固定参数
input_size = 28 * 28
num_classes = 10
#定义超参
batch_size = 64
hidden_size = 128
lr=0.1
momentum = 0.9
epochs = 10

time: 338 μs (started: 2024-11-04 15:43:15 +08:00)


In [14]:
# 构建model
from torch import nn
model=torch.nn.Sequential(nn.Linear(input_size, hidden_size),nn.ReLU(),nn.Linear(hidden_size, num_classes),nn.Softmax(dim=1))


time: 1.92 ms (started: 2024-11-04 15:43:15 +08:00)


In [15]:
def val_eval(model,val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.view(-1, 28 * 28)
            images,labels=images.to(device),labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += len(labels)
            correct += (predicted == labels).sum().item()
    acc=correct/total
    print(f"acc:{acc}")
    return acc

time: 957 μs (started: 2024-11-04 15:43:15 +08:00)


In [16]:
try:
    import optuna
except ModuleNotFoundError:
    !pip install optuna
    import optuna

try:
    import tqdm
except ModuleNotFoundError:
    !pip install tqdm
    import tqdm

time: 687 μs (started: 2024-11-04 15:43:15 +08:00)


In [17]:
def obj(trails):
    kwargs = {
        "hidden_size": trails.suggest_int("hidden_size", 32, 128),
        "batch_size": trails.suggest_int("batch_size", 64, 64),
        "lr": trails.suggest_float("lr", 0.001, 0.1),
        "momentum": trails.suggest_float("momentum", 0.5, 0.9),
        epochs: trails.suggest_int("epochs", 2,10 ),
    }
    model=torch.nn.Sequential(nn.Linear(input_size, kwargs["hidden_size"]),nn.ReLU(),nn.Linear(kwargs["hidden_size"], num_classes),nn.Softmax(dim=1)).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=kwargs["lr"],weight_decay=0.01,momentum=kwargs["momentum"])
    criterion = nn.CrossEntropyLoss()
    train_loader =torch.utils.data.DataLoader(train_data, batch_size=kwargs["batch_size"], shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=kwargs["batch_size"], shuffle=False)
    for epoch in range(kwargs[epochs]):
        for i, (images, labels) in enumerate(train_loader):
            images=images.view(-1,28*28)
            images,labels=images.to(device),labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (i + 1) % 256 == 0:
                print('Epcho [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch, kwargs[epochs], i + 1, len(train_loader), loss.item()))
    return val_eval(model,val_loader)

time: 3.36 ms (started: 2024-11-04 15:43:15 +08:00)


In [18]:
from tqdm import tqdm

class TqdmCallback(object):
    def __init__(self, n_trials):
        self.n_trials = n_trials
        self.pbar = tqdm(total=n_trials)

    def __call__(self, study, trial):
        self.pbar.update(1)

n_trials = 10
tqdm_callback = TqdmCallback(n_trials)
study = optuna.create_study(direction='maximize')
study.optimize(obj, n_trials=n_trials, callbacks=[tqdm_callback])

[I 2024-11-04 15:43:15,384] A new study created in memory with name: no-name-6df1797f-f235-44b5-b9eb-3ed3af85f4c4


Epcho [0/5], Step [256/844], Loss: 1.8234
Epcho [0/5], Step [512/844], Loss: 1.6936
Epcho [0/5], Step [768/844], Loss: 1.7346
Epcho [1/5], Step [256/844], Loss: 1.6231
Epcho [1/5], Step [512/844], Loss: 1.5874
Epcho [1/5], Step [768/844], Loss: 1.6841
Epcho [2/5], Step [256/844], Loss: 1.6381
Epcho [2/5], Step [512/844], Loss: 1.6210
Epcho [2/5], Step [768/844], Loss: 1.5802
Epcho [3/5], Step [256/844], Loss: 1.5984
Epcho [3/5], Step [512/844], Loss: 1.6160
Epcho [3/5], Step [768/844], Loss: 1.6319
Epcho [4/5], Step [256/844], Loss: 1.5426
Epcho [4/5], Step [512/844], Loss: 1.5979
Epcho [4/5], Step [768/844], Loss: 1.6186


[I 2024-11-04 15:43:50,001] Trial 0 finished with value: 0.905 and parameters: {'hidden_size': 84, 'batch_size': 64, 'lr': 0.048158069573203, 'momentum': 0.5800218672518666, 'epochs': 5}. Best is trial 0 with value: 0.905.


acc:0.905
Epcho [0/9], Step [256/844], Loss: 1.8083
Epcho [0/9], Step [512/844], Loss: 1.7627
Epcho [0/9], Step [768/844], Loss: 1.6454
Epcho [1/9], Step [256/844], Loss: 1.8191
Epcho [1/9], Step [512/844], Loss: 1.7014
Epcho [1/9], Step [768/844], Loss: 1.6850
Epcho [2/9], Step [256/844], Loss: 1.5953
Epcho [2/9], Step [512/844], Loss: 1.6513
Epcho [2/9], Step [768/844], Loss: 1.6064
Epcho [3/9], Step [256/844], Loss: 1.6671
Epcho [3/9], Step [512/844], Loss: 1.5993
Epcho [3/9], Step [768/844], Loss: 1.6409
Epcho [4/9], Step [256/844], Loss: 1.6378
Epcho [4/9], Step [512/844], Loss: 1.5856
Epcho [4/9], Step [768/844], Loss: 1.5886
Epcho [5/9], Step [256/844], Loss: 1.5755
Epcho [5/9], Step [512/844], Loss: 1.6198
Epcho [5/9], Step [768/844], Loss: 1.6121
Epcho [6/9], Step [256/844], Loss: 1.5892
Epcho [6/9], Step [512/844], Loss: 1.6596
Epcho [6/9], Step [768/844], Loss: 1.5978
Epcho [7/9], Step [256/844], Loss: 1.6144
Epcho [7/9], Step [512/844], Loss: 1.6433
Epcho [7/9], Step [768/8

[I 2024-11-04 15:44:51,495] Trial 1 finished with value: 0.907 and parameters: {'hidden_size': 117, 'batch_size': 64, 'lr': 0.026192165546679236, 'momentum': 0.622243329796311, 'epochs': 9}. Best is trial 1 with value: 0.907.


...

acc:0.9021666666666667
Epcho [0/2], Step [256/844], Loss: 1.7106
Epcho [0/2], Step [512/844], Loss: 1.8768
Epcho [0/2], Step [768/844], Loss: 1.6719
Epcho [1/2], Step [256/844], Loss: 1.6384
Epcho [1/2], Step [512/844], Loss: 1.5942
Epcho [1/2], Step [768/844], Loss: 1.5762


[I 2024-11-04 15:48:54,869] Trial 8 finished with value: 0.8975 and parameters: {'hidden_size': 63, 'batch_size': 64, 'lr': 0.02604661414070332, 'momentum': 0.8097798962822866, 'epochs': 2}. Best is trial 3 with value: 0.9118333333333334.


acc:0.8975
Epcho [0/7], Step [256/844], Loss: 1.7798
Epcho [0/7], Step [512/844], Loss: 1.6369
Epcho [0/7], Step [768/844], Loss: 1.5939
Epcho [1/7], Step [256/844], Loss: 1.6848
Epcho [1/7], Step [512/844], Loss: 1.6126
Epcho [1/7], Step [768/844], Loss: 1.6369
Epcho [2/7], Step [256/844], Loss: 1.6470
Epcho [2/7], Step [512/844], Loss: 1.6486
Epcho [2/7], Step [768/844], Loss: 1.5892
Epcho [3/7], Step [256/844], Loss: 1.5693
Epcho [3/7], Step [512/844], Loss: 1.6175
Epcho [3/7], Step [768/844], Loss: 1.5964
Epcho [4/7], Step [256/844], Loss: 1.5593
Epcho [4/7], Step [512/844], Loss: 1.6193
Epcho [4/7], Step [768/844], Loss: 1.5616
Epcho [5/7], Step [256/844], Loss: 1.5928
Epcho [5/7], Step [512/844], Loss: 1.6234
Epcho [5/7], Step [768/844], Loss: 1.6238
Epcho [6/7], Step [256/844], Loss: 1.5769
Epcho [6/7], Step [512/844], Loss: 1.6161
Epcho [6/7], Step [768/844], Loss: 1.5871


[I 2024-11-04 15:49:42,254] Trial 9 finished with value: 0.9058333333333334 and parameters: {'hidden_size': 49, 'batch_size': 64, 'lr': 0.09058703178482988, 'momentum': 0.631339603916554, 'epochs': 7}. Best is trial 3 with value: 0.9118333333333334.


acc:0.9058333333333334
time: 6min 26s (started: 2024-11-04 15:43:15 +08:00)


In [19]:
#固定掉超参数
lr= 0.09058703178482988
hidden_size=50
momentum=0.631339603916554
epochs=10 #几个不同epoch差别不太大，多训两轮是不会导致性能明显下降的

time: 1.24 ms (started: 2024-11-04 15:52:30 +08:00)


In [21]:
#使用完整训练集训练并测试pytorch版
train_loader_full = torch.utils.data.DataLoader(train_data_full, batch_size=batch_size, shuffle=True)
model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, num_classes),
    nn.Softmax(dim=1)
).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    model.train()
    for images, labels in train_loader_full:
        images = images.view(-1, 28 * 28).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 1.5631
Epoch [2/10], Loss: 1.5118
Epoch [3/10], Loss: 1.5162
Epoch [4/10], Loss: 1.5068
Epoch [5/10], Loss: 1.5694
Epoch [6/10], Loss: 1.5101
Epoch [7/10], Loss: 1.5532
Epoch [8/10], Loss: 1.5066
Epoch [9/10], Loss: 1.5489
Epoch [10/10], Loss: 1.4948
time: 1min 15s (started: 2024-11-04 16:04:49 +08:00)


In [22]:
#使用测试集测试
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)
acc=val_eval(model,test_loader)
print(f"final acc:{acc}")

acc:0.9546
final acc:0.9546
time: 1.13 s (started: 2024-11-04 16:06:07 +08:00)


In [24]:
#基于myTorch进行构建
#基于myTorch的dataset导入数据
import MyTorch.Dataloader
import numpy as np
#把dataset转为numpy的形式
class trans2MyTensor:
    def __call__(self, img):
        np_img=np.array(img)
        np_img_flat=np_img.flatten()
        #归一化到0 1
        min_val = np.min(np_img_flat)
        max_val = np.max(np_img_flat)
        np_img_flat = (np_img_flat - min_val) / (max_val - min_val)
        result=MyTorch.myTensor(np_img_flat)
        return result
myTensor_transforms = transforms.Compose([
    trans2MyTensor()
])
train_data_full = datasets.MNIST(data_dir, train=True, download=True, transform=myTensor_transforms)
test_data = datasets.MNIST(data_dir, train=False, download=True, transform=myTensor_transforms)

train_dataloader = MyTorch.Dataloader.DataLoader(train_data_full, batch_size=batch_size, shuffle=True)
test_data_loader = MyTorch.Dataloader.DataLoader(test_data, batch_size=batch_size, shuffle=False)

time: 71.6 ms (started: 2024-11-04 16:28:01 +08:00)


In [28]:
from MyTorch import my_nn
import MyTorch.loss_func
import importlib
importlib.reload(MyTorch)
#构建model
model=my_nn.Sequential(
    my_nn.MyLinearLayer(input_size, hidden_size),
    my_nn.ReLU(),
    my_nn.MyLinearLayer(hidden_size, num_classes),
    my_nn.Softmax(dim=1)
)
epochs=1
optimizer = MyTorch.optim.SGD(model.parameters, lr=lr, momentum=momentum)
for epoch in range(epochs):
    for images, labels in train_dataloader:
        outputs = model(images)
        loss = MyTorch.loss_func.CrossEntropyLoss.forward(outputs, labels)
        model.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

ValueError: only one element tensors can be converted to Python scalars

time: 65.6 ms (started: 2024-11-04 16:45:37 +08:00)
